# Instalar dependencias

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%load_ext autoreload
%load_ext tensorboard
%autoreload 2

In [ ]:
!pip install accelerate>=0.12.0 datasets>=1.8.0 sentencepiece!=0.1.92 scipy scikit-learn protobuf torch>=1.3 transformers evaluate

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


## Configurar Dataset

Usaremos como ejemplo el subset en Espanol del dataset [XQuaD](https://github.com/google-deepmind/xquad), disponible en HuggingFace [link text](https://huggingface.co/datasets/google/xquad)

In [34]:
from datasets import load_dataset

dataset = load_dataset("google/xquad","xquad.es")["validation"]

In [35]:
dataset

Dataset({
    features: ['id', 'context', 'question', 'answers'],
    num_rows: 1190
})

In [36]:
from pprint import pprint
pprint(dataset[0])

{'answers': {'answer_start': [133], 'text': ['308']},
 'context': '\ufeffLos Panthers, que además de liderar las intercepciones de '
            'la NFL con 24 y contar con cuatro jugadores de la Pro Bowl, '
            'cedieron solo 308 puntos en defensa y se sitúan en el sexto lugar '
            'de la liga. Kawann Short, tacle defensivo de la Pro Bowl, lideró '
            'al equipo con 11 capturas, 3 balones sueltos forzados y 2 '
            'recuperaciones. A su vez, el liniero Mario Addison, consiguió 6 '
            'capturas y media. En la línea de los Panthers, también destacó '
            'como ala defensiva el veterano Jared Allen ―5 veces jugador de la '
            'Pro Bowl y que fue el líder, en activo, de capturas de la NFL con '
            '136― junto con el también ala defensiva Kony Ealy, que lleva 5 '
            'capturas en solo 9 partidos como titular. Detrás de ellos, Thomas '
            'Davis y Luke Kuechly, dos de los tres apoyadores titulares que '
  

# QA Extractivo o Discriminativo

* La tarea se reduce a clasificar cada palabra como "Respuesta" o "No Respuesta"
* En este ejemplo usaremos un modelo **BERT** finetuned en XQuaD [link text](https://huggingface.co/alon-albalak/bert-base-multilingual-xquad)


In [ ]:
from transformers import pipeline, set_seed

generator = pipeline('question-answering', model='alon-albalak/bert-base-multilingual-xquad')

config.json:   0%|          | 0.00/819 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/667M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

In [ ]:
question = dataset[0]["question"]
context = dataset[0]["context"]

print("Q: ",question)
print("C: ",context)

Q:  ¿Cuántos puntos dejaron escapar en defensa los Panthers?
C:  ﻿Los Panthers, que además de liderar las intercepciones de la NFL con 24 y contar con cuatro jugadores de la Pro Bowl, cedieron solo 308 puntos en defensa y se sitúan en el sexto lugar de la liga. Kawann Short, tacle defensivo de la Pro Bowl, lideró al equipo con 11 capturas, 3 balones sueltos forzados y 2 recuperaciones. A su vez, el liniero Mario Addison, consiguió 6 capturas y media. En la línea de los Panthers, también destacó como ala defensiva el veterano Jared Allen ―5 veces jugador de la Pro Bowl y que fue el líder, en activo, de capturas de la NFL con 136― junto con el también ala defensiva Kony Ealy, que lleva 5 capturas en solo 9 partidos como titular. Detrás de ellos, Thomas Davis y Luke Kuechly, dos de los tres apoyadores titulares que también han sido seleccionados para jugar la Pro Bowl. Davis se hizo con 5 capturas y media, 4 balones sueltos forzados y 4 intercepciones, mientras que Kuechly lideró al equip

In [ ]:
generator(question=question, context=context)

{'score': 0.9970022439956665, 'start': 133, 'end': 136, 'answer': '308'}

## QA Generativo o Abstractivo
* La tarea consiste en generacion texto-hacia-texto
* En este ejemplo usaremos el LLM Qwen 0.5B-Instruct

In [ ]:
from transformers import pipeline, set_seed

llm_generator = pipeline('text-generation', model='Qwen/Qwen2-0.5B-Instruct')

set_seed(42)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
question = dataset[0]["question"]
context = dataset[0]["context"]

print("Q: ",question)
print("C: ",context)

In [ ]:
output = llm_generator(f"""
Descripcion: Reponde brevemente la pregunta usando el texto contexto como base.
Pregunta: {question}
Contexto: {context}
Respuesta:
""", max_new_tokens=50, temperature=0.7)
print(output[0]["generated_text"])


Descripcion: Reponde brevemente la pregunta usando el text contexto como base.
Pregunta: ¿Cuántos puntos dejaron escapar en defensa los Panthers?
Contexto: ﻿Los Panthers, que además de liderar las intercepciones de la NFL con 24 y contar con cuatro jugadores de la Pro Bowl, cedieron solo 308 puntos en defensa y se sitúan en el sexto lugar de la liga. Kawann Short, tacle defensivo de la Pro Bowl, lideró al equipo con 11 capturas, 3 balones sueltos forzados y 2 recuperaciones. A su vez, el liniero Mario Addison, consiguió 6 capturas y media. En la línea de los Panthers, también destacó como ala defensiva el veterano Jared Allen ―5 veces jugador de la Pro Bowl y que fue el líder, en activo, de capturas de la NFL con 136― junto con el también ala defensiva Kony Ealy, que lleva 5 capturas en solo 9 partidos como titular. Detrás de ellos, Thomas Davis y Luke Kuechly, dos de los tres apoyadores titulares que también han sido seleccionados para jugar la Pro Bowl. Davis se hizo con 5 capturas 

### Few-shot

Definimos los ejemplos a incluir en el prompt

In [ ]:
shot1_q = dataset[100]["question"]
shot1_c = dataset[100]["context"]
shot1_a = " ".join(dataset[100]["answers"]["text"])

shot2_q = dataset[200]["question"]
shot2_c = dataset[200]["context"]
shot2_a = " ".join(dataset[200]["answers"]["text"])

print("Q1: ",shot1_q)
print("C1: ",shot1_c)
print("A1: ",shot1_a)
print()
print("Q2: ",shot2_q)
print("C2: ",shot2_c)
print("A2: ",shot2_a)

Q1:  ¿Quién fue el principal enemigo de los normandos en Italia, el Imperio bizantino y Armenia?
C1:  Los normandos comenzaron a adentrarse en el Imperio bizantino y más tarde en Armenia, poco después de que lo hicieran en Italia. Lucharon contra pechenegos, protobúlgaros, y sobretodo contra los turcos selyúcidas. En un primer momento, fueron los lombardos quienes alentaron a los mercenarios normandos a dirigirse hacia el sur para actuar contra los bizantinos, pero pronto acabaron peleando en el servicio de estos últimos en Sicilia. Junto con los contingentes de varegos y lombardos, fueron una fuerza destacada en la campaña siciliana de Jorge Maniaces (1038-1040). Si los normandos en el servicio griego procedían o no de la Italia normanda, es una cuestión a debatir; pero, lo que parece más probable es que solo unos pocos vinieron de esa zona. También se desconoce cuántos de los francos, como los llamaban los bizantinos, eran normandos y no otros franceses.
A1:  turcos selyúcidas

Q2:  

Armamos el prompt

In [ ]:
prompt = f"""Descripcion: Reponde brevemente la pregunta usando el texto contexto como base.
Pregunta: {shot1_q}
Contexto: {shot1_c}
Respuesta: {shot1_a}

Pregunta: {shot2_q}
Contexto: {shot2_c}
Respuesta: {shot2_a}

Pregunta: {question}
Contexto: {context}
Respuesta:"""

print(prompt)

Descripcion: Reponde brevemente la pregunta usando el texto contexto como base.
Pregunta: ¿Quién fue el principal enemigo de los normandos en Italia, el Imperio bizantino y Armenia?
Contexto: Los normandos comenzaron a adentrarse en el Imperio bizantino y más tarde en Armenia, poco después de que lo hicieran en Italia. Lucharon contra pechenegos, protobúlgaros, y sobretodo contra los turcos selyúcidas. En un primer momento, fueron los lombardos quienes alentaron a los mercenarios normandos a dirigirse hacia el sur para actuar contra los bizantinos, pero pronto acabaron peleando en el servicio de estos últimos en Sicilia. Junto con los contingentes de varegos y lombardos, fueron una fuerza destacada en la campaña siciliana de Jorge Maniaces (1038-1040). Si los normandos en el servicio griego procedían o no de la Italia normanda, es una cuestión a debatir; pero, lo que parece más probable es que solo unos pocos vinieron de esa zona. También se desconoce cuántos de los francos, como los l

In [ ]:
output = llm_generator(prompt, max_new_tokens=5, temperature=0.7)
print(output[0]["generated_text"])

Descripcion: Reponde brevemente la pregunta usando el texto contexto como base.
Pregunta: ¿Quién fue el principal enemigo de los normandos en Italia, el Imperio bizantino y Armenia?
Contexto: Los normandos comenzaron a adentrarse en el Imperio bizantino y más tarde en Armenia, poco después de que lo hicieran en Italia. Lucharon contra pechenegos, protobúlgaros, y sobretodo contra los turcos selyúcidas. En un primer momento, fueron los lombardos quienes alentaron a los mercenarios normandos a dirigirse hacia el sur para actuar contra los bizantinos, pero pronto acabaron peleando en el servicio de estos últimos en Sicilia. Junto con los contingentes de varegos y lombardos, fueron una fuerza destacada en la campaña siciliana de Jorge Maniaces (1038-1040). Si los normandos en el servicio griego procedían o no de la Italia normanda, es una cuestión a debatir; pero, lo que parece más probable es que solo unos pocos vinieron de esa zona. También se desconoce cuántos de los francos, como los l